In [3]:
!pip install google_streetview

You should consider upgrading via the '/Users/vdefont/miniconda3/bin/python -m pip install --upgrade pip' command.


In [4]:
import numpy as np
import pandas as pd
import torch
import google_streetview.api
import random
from pathlib import Path
import os
import shutil
from shutil import copyfile
import re

In [9]:
data = pd.read_csv("data.csv")

In [ ]:
# NOTE: This script uses an intermediate directory "scrape", since the API likes
# to scrape into an empty dir each time

In [15]:
def get_img_paths(dest_dir, btm, top):
    paths = [dest_dir/f"gsv_{i}.jpg" for i in range(top-btm)]
    return [str(p) if p.exists() else None for p in paths]

In [16]:
def scrape(btm, top):
    dest_dir = Path(f"scrape/{btm}_{top}")
    dest_dir.mkdir(exist_ok=False)

    to_scrape = data.iloc[data.index[btm:top].tolist()]
    print("to_scrape")
    print(to_scrape)

    headings = [random.randint(0, 359) for _i in range(len(to_scrape))]
    print("headings")
    print(headings)

    params = [
        {
            'size': '1000x1000',
            'location': f'{lat},{lng}',
            'heading': str(heading),
            'pitch': '0',
            'fov': '105',
            'key': '<KEY>',
            'source': 'outdoor',
        }
        for lat, lng, heading in zip(to_scrape.lat, to_scrape.lng, headings)
    ]
    results = google_streetview.api.results(params)
    print("GOT RESULTS")
    results.download_links(dest_dir)

    metadata = pd.DataFrame({'index': to_scrape.index, 'id': to_scrape.id, 'heading': headings})
    metadata.to_csv(f'scrape/metadata/{btm}_{top}')

    for i, country, path in zip(to_scrape.index, to_scrape.country, get_img_paths(dest_dir, btm, top)):
        if path is not None:
            copyfile(path, f"all/{i}.jpg")

In [7]:
def make_intervals(btm, top, step):
    ret = list(zip(range(btm, top, step), range(btm+step, top+1, step)))
    # Cover the remainder
    if len(ret) > 0 and ret[-1][1] < top:
        ret.append((ret[-1][1], top))
    return ret

In [10]:
intervals = make_intervals(0, len(data), 250)

In [19]:
curset = "train"
for btm, top in intervals:
    print(f"\n\nScraping {curset} from {btm} to {top}")
    %time scrape(curset, btm, top)



Scraping test from 4500 to 4750
to_scrape
                           id        lat        lng country  population  \
44931  nES_QGeOPBhEBQVT5bxfuw -10.476357 -36.439137      br    0.239216   
44934  9ejYgyUwdV4Cb7SFyjiAUA  -9.910189 -47.993313      br    0.517647   
44976  dsTaSTdzEbYCBY8myzCYOw -19.721605 -40.021538      br    0.215686   
45006  SKe0TY4leVPdCITq9pcPGw  -0.627264 -47.361300      br    0.400000   
45016  AmGWyH3SW8lp5YeqUy1zng -22.508894 -48.557888      br    0.388235   
...                       ...        ...        ...     ...         ...   
47373  ah_5VEUEiW_IiEGmhAqSFg  53.139490   7.035398      nl    0.392157   
47377  oWNVcSivhSE-WhKxSk9XDQ  53.229954   6.572316      nl    0.603922   
47378  X1mVQyw4x5c0gVmBf0TLMw  52.296590   5.237664      nl    0.600000   
47410  2z0B6ubQ7zcdwwkSHwu12w  51.616116   4.746836      nl    0.600000   
47422  -94tkQEcmv2AF9z5-K717g  50.866500   5.652889      nl    0.572549   

      country_name  
44931       Brazil  
44934       B